In [2]:
import os
import torch
import torchaudio
import soundfile as sf
import pickle

from speechbrain.pretrained import SpeakerRecognition


verification_model = SpeakerRecognition.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_ecapa"
)


/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoi

hyperparams.yaml: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/hyperparams.yaml' -> '/content/pretrained_ecapa/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered che

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/embedding_model.ckpt' -> '/content/pretrained_ecapa/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /content/pretrained_ecapa/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


...

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/mean_var_norm_emb.ckpt' -> '/content/pretrained_ecapa/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /content/pretrained_ecapa/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/classifier.ckpt' -> '/content/pretrained_ecapa/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /content/pretrained_ecapa/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/label_encoder.txt' -> '/content/pretrained_ecapa/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /content/pretrained_ecapa/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /content/pretrained_ecapa/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /content/pretrained_ecapa/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /content/pretrained_ecapa/classifier.ckpt
DEBUG:speechbrain.utils.

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import numpy as np
import random
import glob
import os
from torch.utils.data import Dataset, DataLoader
from speechbrain.inference.speaker import SpeakerRecognition

LIBRISPEECH_PATH = "/content/lib/LibriSpeech/dev-clean/"
NOISE_PATH = "/content/esc/ESC-50-master/audio/"
LOCAL_MODEL_PATH = "pretrained_ecapa"
MODEL_SAVE_PATH = "speaker_model_noise_robust.pth"

NUM_EPOCHS = 5
BATCH_SIZE = 16
LEARNING_RATE = 1e-5
MIN_SNR_DB = 0
MAX_SNR_DB = 15

try:
    print(f"[INFO] Loading ECAPA-TDNN model from LOCAL folder: {LOCAL_MODEL_PATH}")
    inference_model = SpeakerRecognition.from_hparams(source=LOCAL_MODEL_PATH)

    feature_extractor = inference_model.mods.compute_features
    embedding_model = inference_model.mods.embedding_model
    trainable_model = nn.Sequential(feature_extractor, embedding_model)

    print("[INFO] Trainable model components extracted successfully!")

except Exception as e:
    print(f"[ERROR] Could not load or extract model: {e}")
    trainable_model = None

def mix_with_noise(waveform, noise_waveform, snr_db):
    sig_power = waveform.norm(p=2)
    noise_power = noise_waveform.norm(p=2)
    snr = 10 ** (snr_db / 20)
    scale = snr * noise_power / (sig_power + 1e-8)
    noisy_waveform = (scale * waveform + noise_waveform) / 2
    return noisy_waveform

class NoisySpeakerTripletDataset(Dataset):
    def __init__(self, speaker_files, speaker_map, speakers, noise_files):
        self.speaker_files = speaker_files
        self.speaker_map = speaker_map
        self.speakers = speakers
        self.noise_files = noise_files

    def __len__(self):
        return len(self.speaker_files)

    def _load_and_process_wav(self, file_path, target_len=48000):
        wav, sr = torchaudio.load(file_path)
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)
        if sr != 16000:
             wav = torchaudio.functional.resample(wav, sr, 16000)

        if wav.shape[1] > target_len:
            wav = wav[:, :target_len]
        else:
            wav = torch.nn.functional.pad(wav, (0, target_len - wav.shape[1]))
        return wav

    def __getitem__(self, index):
        anchor_file = self.speaker_files[index]
        anchor_speaker = self.speaker_map[anchor_file]

        positive_list = [f for f, s in self.speaker_map.items() if s == anchor_speaker and f != anchor_file]
        positive_file = random.choice(positive_list) if positive_list else anchor_file

        negative_speaker = random.choice([s for s in self.speakers if s != anchor_speaker])
        negative_list = [f for f, s in self.speaker_map.items() if s == negative_speaker]
        negative_file = random.choice(negative_list)

        anchor_wav = self._load_and_process_wav(anchor_file)
        positive_wav = self._load_and_process_wav(positive_file)
        negative_wav = self._load_and_process_wav(negative_file)

        noise_file = random.choice(self.noise_files)
        noise_wav = self._load_and_process_wav(noise_file, target_len=anchor_wav.shape[1])

        snr_db = random.uniform(MIN_SNR_DB, MAX_SNR_DB)
        noisy_anchor = mix_with_noise(anchor_wav, noise_wav, snr_db)
        noisy_positive = mix_with_noise(positive_wav, noise_wav, snr_db)
        noisy_negative = mix_with_noise(negative_wav, noise_wav, snr_db)

        return noisy_anchor, noisy_positive, noisy_negative

if __name__ == "__main__":
    if trainable_model:
        speaker_files = glob.glob(f"{LIBRISPEECH_PATH}/*/*/*.flac")
        speaker_map = {file: file.split('/')[-3] for file in speaker_files}
        speakers = list(set(speaker_map.values()))
        noise_files = glob.glob(f"{NOISE_PATH}/*.wav")

        if not speaker_files or not noise_files:
            raise RuntimeError("Could not find speaker or noise files. Check your paths.")

        triplet_dataset = NoisySpeakerTripletDataset(speaker_files, speaker_map, speakers, noise_files)
        train_loader = DataLoader(triplet_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

        triplet_loss = nn.TripletMarginLoss(margin=0.2, p=2)
        optimizer = optim.Adam(trainable_model.parameters(), lr=LEARNING_RATE)
        scaler = torch.amp.GradScaler(enabled=torch.cuda.is_available())

        device = "cuda" if torch.cuda.is_available() else "cpu"
        trainable_model.to(device)

        for param in trainable_model.parameters():
            param.requires_grad = True

        trainable_model.train()

        print("\n[INFO] Starting fine-tuning for noise robustness...")
        for epoch in range(NUM_EPOCHS):
            total_loss = 0
            for i, (anchor, positive, negative) in enumerate(train_loader):
                anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

                anchor = anchor.squeeze(1)
                positive = positive.squeeze(1)
                negative = negative.squeeze(1)

                optimizer.zero_grad(set_to_none=True)

                with torch.amp.autocast(device_type=device, enabled=torch.cuda.is_available()):
                    emb_a = trainable_model(anchor)
                    emb_p = trainable_model(positive)
                    emb_n = trainable_model(negative)
                    loss = triplet_loss(emb_a, emb_p, emb_n)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                total_loss += loss.item()

                if (i + 1) % 10 == 0:
                    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

            print(f"--- End of Epoch {epoch+1}, Average Loss: {total_loss / len(train_loader):.4f} ---")

        print("[INFO] Fine-tuning complete.")
        torch.save(embedding_model.state_dict(), MODEL_SAVE_PATH)
        print(f"[INFO] Noise-robust model saved to {MODEL_SAVE_PATH}")


[INFO] Loading ECAPA-TDNN model from LOCAL folder: pretrained_ecapa


DEBUG:speechbrain.utils.parameter_transfer:Fetching files for pretraining (no collection directory set)
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
D

[INFO] Trainable model components extracted successfully!

[INFO] Starting fine-tuning for noise robustness...
Epoch [1/20], Step [10/169], Loss: 0.0000
Epoch [1/20], Step [20/169], Loss: 0.0000
Epoch [1/20], Step [30/169], Loss: 1.1983
Epoch [1/20], Step [40/169], Loss: 0.0000
Epoch [1/20], Step [50/169], Loss: 0.0000
Epoch [1/20], Step [60/169], Loss: 1.0006
Epoch [1/20], Step [70/169], Loss: 0.0000
Epoch [1/20], Step [80/169], Loss: 0.0000
Epoch [1/20], Step [90/169], Loss: 0.0000
Epoch [1/20], Step [100/169], Loss: 0.0000
Epoch [1/20], Step [110/169], Loss: 0.0000
Epoch [1/20], Step [120/169], Loss: 0.0000
Epoch [1/20], Step [130/169], Loss: 1.3290
Epoch [1/20], Step [140/169], Loss: 0.0000
Epoch [1/20], Step [150/169], Loss: 0.0000
Epoch [1/20], Step [160/169], Loss: 0.0000
--- End of Epoch 1, Average Loss: 0.3297 ---
Epoch [2/20], Step [10/169], Loss: 0.0000
Epoch [2/20], Step [20/169], Loss: 0.4588
Epoch [2/20], Step [30/169], Loss: 0.0000
Epoch [2/20], Step [40/169], Loss: 0.00